In [ ]:
!pip install numpy
!pip install pandas
!pip install catboost
!pip install imblearn

Загрузим данные и библиотеки

In [237]:
from warnings import filterwarnings

import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler

df = pd.read_parquet('END_minmax_rev.parquet')
df_test = pd.read_parquet('SUB.parquet')

add = pd.read_parquet('revenue.parquet')
add_test = pd.read_parquet('revenue_1.parquet')

Удалим лишние столбцы и получим размерность данных

In [238]:
df.drop(['revenue','toilet','weekday_h','Free-standing'],axis=1,inplace=True)


add.drop('buy_post',axis=1,inplace=True)

df.shape, df_test.shape

Добавим новые данные в датасет, обработаем его и разделим их

In [246]:
df = pd.merge(df, add, on='customer_id', how='inner')


df.drop('revenue_x',axis=1,inplace=True)
df['s/p']=df['revenue_y']/df['purchase_count']

df.average_time_between_purchases.fillna(0,inplace=True)
df.std_customer.fillna(0, inplace=True)

df.drop(['dish_name', 'format_name', 'revenue_delta_class', 'most_common_pattern', 'customer_id'], axis = 1, inplace = True)

X, y = df.drop(['date_diff_post', 'buy_post'], axis=1), df['buy_post']

Обучим CatBoost модель

In [255]:
filterwarnings('ignore')

oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
Xtrain, ytrain = oversampler.fit_resample(X, y)

model = CatBoostClassifier(task_type='GPU',random_state = 42)
model.fit(Xtrain,ytrain)

Learning rate set to 0.023487
0:	learn: 0.6890369	total: 26.7ms	remaining: 26.7s
1:	learn: 0.6851561	total: 52.2ms	remaining: 26s
2:	learn: 0.6814491	total: 77.9ms	remaining: 25.9s
3:	learn: 0.6779871	total: 104ms	remaining: 25.8s
4:	learn: 0.6746445	total: 129ms	remaining: 25.6s
5:	learn: 0.6714365	total: 153ms	remaining: 25.4s
6:	learn: 0.6684538	total: 179ms	remaining: 25.4s
7:	learn: 0.6656124	total: 204ms	remaining: 25.3s
8:	learn: 0.6629377	total: 230ms	remaining: 25.4s
9:	learn: 0.6603261	total: 254ms	remaining: 25.2s
10:	learn: 0.6578839	total: 279ms	remaining: 25.1s
11:	learn: 0.6554931	total: 303ms	remaining: 25s
12:	learn: 0.6533235	total: 328ms	remaining: 24.9s
13:	learn: 0.6511759	total: 352ms	remaining: 24.8s
14:	learn: 0.6492141	total: 376ms	remaining: 24.7s
15:	learn: 0.6472229	total: 401ms	remaining: 24.6s
16:	learn: 0.6453713	total: 426ms	remaining: 24.6s
17:	learn: 0.6436388	total: 453ms	remaining: 24.7s
18:	learn: 0.6420047	total: 480ms	remaining: 24.8s
19:	learn: 0

Вывод наиболее релевантных фич

In [263]:
pd.DataFrame({'cols':X.columns ,'imp':model.feature_importances_}).sort_values('imp', ascending=False)

,cols,imp
12,total_time_between_purchases,25.537464
28,revenue_y,16.822110
13,average_time_between_purchases,12.748992
4,month_h,10.535109
29,s/p,4.497735
18,7_burger,2.479458
0,ownareaall_sqm,2.276692
5,quarter_h,2.164572
24,5_fri,1.745490
21,6_snacks,1.672834


Обаботка тестового датасета, сохранение предсказаний теста


In [258]:
df_test.drop(['revenue','toilet','weekday_h','Free-standing'],axis=1,inplace=True)
df_test['revenue']=add_test['revenue']
df_test['s/p']=df['revenue_y']/df['purchase_count']
df_test.drop(['dish_name', 'format_name', 'revenue_delta_class', 'most_common_pattern', 'customer_id'], axis = 1, inplace = True)

pred = model.predict(df_test)

sub = pd.read_csv('submission.csv',sep=';')
sub.customer_id = pd.read_parquet('SUB.parquet').customer_id
sub.buy_post = pred
sub.to_csv('sub_pred.csv',sep=';',index=False)